# POSNNET use case example

Welcome to this POSNNET use case example. This notebook presents an example of how to use the framework in a real-world scenario. It is a complementary resource to the user guide notebook.


**Here is the use case:**

Tony and Johnny developed a device designed for runners to analyze their training sessions and competitions. The device is capable of outputting both position and velocity data. These outputs are then processed to derive various performance metrics for the runner. Since runners typically analyze their sessions after completing them, postprocessing is a suitable solution rather than requiring real-time computation.

<u>Device description</u>

The device is equipped with:

- An **Inertial Measurement Unit (IMU)**, consisting of an accelerometer and a gyroscope, both providing data across three axes at a frequency of 100 Hz.
- A **GPS chip**, which provides velocity data across three axes at a frequency of 10 Hz.
- A **data fusion algorithm**, which combines the IMU and GPS data to output final position and velocity at a frequency of 100 Hz.

<u>Problem observed</u>

Some users experience GPS outages during usage. Because the output position and velocity depend heavily on GPS data, the accuracy of the results drops significantly during these outages. Tony and Johnny have identified two main scenarios where GPS outages occur:

- **At the beginning of a session**, when the user starts indoors. In this case, GPS outages last between 10 and 30 seconds.
- **Within a session**, when the user passes through areas with poor GPS coverage (e.g., forests or mountains). These outages last between 20 seconds and 10 minutes.

Additionally, they observed that when multiple GPS outages occur within a session, there is always at least 10 seconds of operational GPS data between them.

<u>Solution</u>

They decided to use the POSNNET framework, which is particularly well-suited for their problem. To maximize efficiency and meet both short-term and long-term goals, they chose to work in parallel:

- **Tony** will run the framework with minimal parameters to quickly obtain a usable solution. This version is intended to provide immediate improvements and be deployed to clients who are dissatisfied with the loss of accuracy during GPS outages.

- **Johnny** will run the framework using more robust parameters to produce a more accurate and stable solution. Once finalized, Johnny’s solution will replace Tony’s temporary version in production.

Both Tony and Johnny have access to a system equipped with a suitable GPU capable of handling the framework’s computational requirements. Tony will use a **dedicated computing node** to perform the tuning, ensuring that all resources are available exclusively for his tasks. Johnny, on the other hand, will use a **shared computing node**, meaning that the available resources may be distributed among multiple users.

<u>Note</u> 

Tony and Johnny are used in this example to illustrate two different usage scenarios:

- **Tony** represents a case where you have limited time and/or low computational resources.  
- **Johnny** represents a case where you have plenty of time and/or access to a powerful computing system.

These examples help demonstrate how to adapt the use of the POSNNET framework based on available resources and urgency.


## 1. Data collection

Before starting with the POSNNET framework, Tony and Johnny aggregated data from multiple devices used by runners who agreed to participate in the data collection. The recorded sessions cover a wide range of usage scenarios for their device:

- **Environments:** urban areas, forests, flat courses, and routes with varying elevation (ascents and descents)  
- **Runner profiles:** beginner, intermediate, and advanced levels  
- **Session characteristics:** varying in intensity and duration

After aggregating the data, they conducted a **quality check** to filter out sessions where the GPS signal was not consistently strong throughout the entire duration. After this filtering process, they retained **100 hours of high-quality data**.

They then split the dataset into three parts:
- **70 hours for training**
- **15 hours for validation**
- **15 hours for evaluation**

The split was done carefully to ensure that all three subsets maintained a representative distribution of the various usage scenarios.

## 2. Project creation

Tony and Johnny each create a project.

In [ ]:
from posnnet import Project

project_tony = Project(project_name='tony')
project_johnny = Project(project_name='johnny')

## 3. Settings

### 3.1. General settings

**Random seed**

The value of the random seed has no impact on the results. To ensure they work on the same dataset and obtain reproducible results, both Tony and Johnny will use the same value.

**Dtype**

Tony can consider using <code>'float16'</code> or <code>'float32'</code>. Since he wants quick results but also wants to avoid sacrificing too much accuracy, he will stick to <code>'float32'</code> and uses the mixed precision which will run in both 16 and 32 bits precision depending of the operations.

Johnny can consider using <code>'float32'</code> or <code>'float64'</code>. While <code>'float64'</code> could slightly improve accuracy, it comes with a significant computational cost. Johnny estimates that the minor accuracy gain does not justify the additional cost, so he will also use <code>'float32'</code>.

**Device**

Both Tony and Johnny will use <code>'auto'</code> to let the framework automatically select the most suitable device available (preferably a GPU).

In [ ]:
project_tony.set_general_settings(
    random_seed=42,
    dtype='float32',
    device='auto'
)

project_johnny.set_general_settings(
    random_seed=42,
    dtype='float32',
    device='auto'
)

### 3.2. CSV files settings

**CSV sep**

They both work on the same CSV files, which use <code>','</code> as the separator.

**CSV encoding**

They both work on the same CSV files, which use <code>'utf-8'</code> as the encoding format.

In [ ]:
project_tony.set_csv_files_settings(
    csv_sep=',',
    csv_encoding='utf-8'
)

project_johnny.set_csv_files_settings(
    csv_sep=',',
    csv_encoding='utf-8'
)

### 3.3. Sensor settings

They both use the same sensor, so the sensor settings are identical for both cases.

**Frequency**

The frequency of the data provided by the sensor is <code>100</code> Hz. Although the GPS chip itself provides data at 10 Hz, each measurement is duplicated 10 times to align with the 100 Hz frequency of the sensor output.

**Features names**

The IMU provides:
- 3D acceleration from the accelerometer, named <code>accelerometer_i</code> where <code>i</code> is either <code>x</code>, <code>y</code>, or <code>z</code>
- 3D angular velocity from the gyroscope, named <code>gyroscope_i</code> where <code>i</code> is either <code>x</code>, <code>y</code>, or <code>z</code>
- No magnetic field data (the sensor does not include a magnetometer)

The GPS chip provides:
- 3D velocity, named <code>velocity_gps_i</code> where <code>i</code> is either <code>x</code>, <code>y</code>, or <code>z</code>

The fusion algorithm provides:
- 3D position, named <code>position_fusion_i</code> where <code>i</code> is either <code>x</code>, <code>y</code>, or <code>z</code>
- 3D velocity, named <code>velocity_fusion_i</code> where <code>i</code> is either <code>x</code>, <code>y</code>, or <code>z</code>
- No orientation data


In [ ]:
project_tony.set_sensor_settings(
    frequency=100,
    name_accelerometer_x='accelerometer_x',
    name_accelerometer_y='accelerometer_y',
    name_accelerometer_z='accelerometer_z',
    name_gyroscope_x='gyroscope_x',
    name_gyroscope_y='gyroscope_y',
    name_gyroscope_z='gyroscope_z',
    name_magnetometer_x=None,
    name_magnetometer_y=None,
    name_magnetometer_z=None,
    name_velocity_gps_x='velocity_gps_x',
    name_velocity_gps_y='velocity_gps_y',
    name_velocity_gps_z='velocity_gps_z',
    name_position_fusion_x='position_fusion_x',
    name_position_fusion_y='position_fusion_y',
    name_position_fusion_z='position_fusion_z',
    name_velocity_fusion_x='velocity_fusion_x',
    name_velocity_fusion_y='velocity_fusion_y',
    name_velocity_fusion_z='velocity_fusion_z',
    name_orientation_fusion_x=None,
    name_orientation_fusion_y=None,
    name_orientation_fusion_z=None
)

project_johnny.set_sensor_settings(
    frequency=100,
    name_accelerometer_x='accelerometer_x',
    name_accelerometer_y='accelerometer_y',
    name_accelerometer_z='accelerometer_z',
    name_gyroscope_x='gyroscope_x',
    name_gyroscope_y='gyroscope_y',
    name_gyroscope_z='gyroscope_z',
    name_magnetometer_x=None,
    name_magnetometer_y=None,
    name_magnetometer_z=None,
    name_velocity_gps_x='velocity_gps_x',
    name_velocity_gps_y='velocity_gps_y',
    name_velocity_gps_z='velocity_gps_z',
    name_position_fusion_x='position_fusion_x',
    name_position_fusion_y='position_fusion_y',
    name_position_fusion_z='position_fusion_z',
    name_velocity_fusion_x='velocity_fusion_x',
    name_velocity_fusion_y='velocity_fusion_y',
    name_velocity_fusion_z='velocity_fusion_z',
    name_orientation_fusion_x=None,
    name_orientation_fusion_y=None,
    name_orientation_fusion_z=None
)

### 3.4. Dataset settings

**Number of duplications per evaluation session**

Tony and Johnny have recorded up to 100 hours of high-quality data, with 15 hours allocated for evaluation. Given the sufficient size of the evaluation dataset, they will use <code>'normal'</code> for the number of duplications per evaluation session.

**Sessions id**

Tony and Johnny split their dataset into three parts while maintaining a good distribution of usage scenarios, using the following proportions:
- 70% for training
- 15% for validation
- 15% for evaluation

NB: The dataset parameters are essentially the same for both Tony and Johnny.

In [ ]:
# The purpose of this notebook is purely demonstrative, so only a small portion of the dataset has been included to speed up operations.

project_tony.set_dataset_settings(
    n_duplications_per_eval_session='normal',
    sessions_id_training=[1, 2, 3, 4, 5, 6],
    sessions_id_validation=[7, 8],
    sessions_id_evaluation=[9, 10]
)

project_johnny.set_dataset_settings(
    n_duplications_per_eval_session='normal',
    sessions_id_training=[1, 2, 3, 4, 5, 6],
    sessions_id_validation=[7, 8],
    sessions_id_evaluation=[9, 10]
)

### 3.5. Objective settings

**Number of minimum seconds of operational GPS**

As mentioned in the introduction, preliminary analysis has shown that at least 10 seconds of operational GPS can be guaranteed around each GPS outage. Since the framework can operate with as few as 4 seconds, both Tony and Johnny will use <code>4</code>.

**GPS outage durations evaluation at beginning**

As stated earlier, the device is subject to GPS outages at the beginning of sessions, typically lasting from 10 to 30 seconds. Both Tony and Johnny will use: <code>[("00:10", "00:30")]</code>

**GPS outage durations evaluation within**

Preliminary analysis also revealed that GPS outages can occur within sessions, lasting from 20 seconds up to 10 minutes. To allow fine-grained evaluation across different outage durations, they will split the range into several segments: <code>[("00:20", "01:00"), ("01:00", "03:00"), ("03:00", "06:00"), ("06:00", "10:00")]</code>

**GPS outage durations evaluation at end**

Since the analysis did not reveal any GPS outages occurring at the end of sessions, they will use:  
<code>[]</code>

In [ ]:
project_tony.set_objective_settings(
    n_minimum_seconds_operational_gps=4,
    gps_outage_durations_eval_at_beginning=[("00:10", "00:30")],
    gps_outage_durations_eval_within=[("00:20", "01:00"), ("01:00", "03:00"), ("03:00", "06:00"), ("06:00", "10:00")],
    gps_outage_durations_eval_at_end=[]
)

project_johnny.set_objective_settings(
    n_minimum_seconds_operational_gps=4,
    gps_outage_durations_eval_at_beginning=[("00:10", "00:30")],
    gps_outage_durations_eval_within=[("00:20", "01:00"), ("01:00", "03:00"), ("03:00", "06:00"), ("06:00", "10:00")],
    gps_outage_durations_eval_at_end=[]
)

### 3.6. Training settings

**Coeff sampling training**

To determine an appropriate coefficient value, Tony and Johnny considered the following:
- Their dataset is large enough, so a lower coefficient is not needed to artificially "increase" the dataset size.
- Their device outputs data at a frequency of 100 Hz.
- They are targeting GPS outages lasting from 10 seconds to 10 minutes.

Based on this, both will use a coefficient of <code>10000</code>. This means that for each session, a data window (up to 10 minutes long) will be cut every 10000 / 100 Hz = 100 seconds.

**Coeff sampling validation**

Following the framework's recommendation to use one-fifth of the training coefficient, they will both use <code>2000</code> for validation.

**Number of epochs**

- Tony, aiming for quick results, will use <code>30</code> epochs.
- Johnny, prioritizing accuracy, will extend training up to <code>100</code> epochs.

**Patience**

- Tony will use <code>8</code> epochs for early stopping.
- Johnny will use <code>10</code> epochs to allow a bit more flexibility before stopping.

**Number of epochs for the sampling**

To follow best practices (one-fifth of the maximum number of epochs, with a minimum of 10):
- Tony will use <code>10</code>.
- Johnny will use <code>20</code>.

**Velocity loss**

Both will use <code>'mse'</code> to prioritize minimizing extreme errors.

**Mixed precision**

Both will enable mixed precision by setting it to <code>True</code>, taking advantage of faster computation and reduced memory usage, with minimal/none loss in accuracy.

**Num workers**

Each will use <code>4</code> workers, which is recommended for one GPU.

**Number of epochs between training checkpoint**

- Tony, using a **dedicated computing node** where interruptions are not expected, will set this to the maximum number of epochs (<code>30</code>). This means no checkpoints will be saved.
- Johnny, using a **shared computing node** where interruptions may occur, will use <code>20</code>. This will allow the framework to save up to 4 checkpoints (at epochs 20, 40, 60, and 80) during a full 100-epoch training.


In [ ]:
project_tony.set_training_settings(
    coeff_sampling_training=10000,
    coeff_sampling_validation=2000,
    n_epochs_sampling=10,
    n_epochs=30,
    patience=8,
    num_workers=4,
    use_mixed_precision=True,
    n_epochs_training_checkpoint=30
)

project_johnny.set_training_settings(
    coeff_sampling_training=10000,
    coeff_sampling_validation=2000,
    n_epochs_sampling=20,
    n_epochs=100,
    patience=10,
    num_workers=4,
    use_mixed_precision=True,
    n_epochs_training_checkpoint=20
)

### 3.7. Tuning settings

**N startup trials**

- Tony plans to perform between 30 to 50 trials per configuration. To ensure quick results while still exploring the hyperparameter space, he will set <code>10</code> startup trials. These will be randomly sampled before the framework switches to Bayesian optimization.

- Johnny plans to perform 100 trials per configuration. To allow better exploration of the hyperparameter space before Bayesian optimization begins, he will set <code>20</code> startup trials.

**Save every state dict**

Since neither Tony nor Johnny need to debug or have any specific requirement to keep every model state, they will only save the best-performing state dict by setting <code>False</code>.

In [ ]:
project_tony.set_tuning_settings(
    n_startup_trials=10,
    save_every_state_dicts=False
)

project_johnny.set_tuning_settings(
    n_startup_trials=20,
    save_every_state_dicts=False
)

### 3.8. Settings validation

Finally, Tony and Johnny can validate their project settings.

In [ ]:
project_tony.validate_settings()

In [ ]:
project_johnny.validate_settings()

Upon validation, both receive the following warnings:

- **3 warnings** for not providing names for the magnetometer axes — this is expected and acceptable, as their IMU does not include a magnetometer.
- **3 warnings** for not providing names for the fusion orientation axes — also expected, since their fusion algorithm does not output orientation data.

In addition, Tony receives an extra warning:

- **1 warning** indicating that using 30 epochs as the maximum of training epochs and setting 30 epochs between each training checkpoint means no training checkpoints will be saved. This is intentional and aligns with Tony’s plan, so it is not a concern.

In conclusion, Tony and Johnny decide not to modify their settings in response to the warnings, as all of them are expected and justified. Therefore, they proceed by calling the validation method again with <code>force_validation=True</code> to finalize and validate their settings.

In [ ]:
project_tony.validate_settings(force_validation=True)

In [ ]:
project_johnny.validate_settings(force_validation=True)

## 4. Preprocess data

Now that Tony and Johnny have configured their respective projects, they can proceed with the data preprocessing stage.

In [ ]:
project_tony.preprocess_data(verbosity=True)

In [ ]:
project_johnny.preprocess_data(verbosity=True)

## 5. Scalers fitting

Before starting to tune the configurations on their data, they have to fit the scalers.

In [ ]:
project_tony.fit_scalers()

In [ ]:
project_johnny.fit_scalers()

## 6. Models tuning and evaluation

Because Tony and Johnny have different goals and levels of urgency (as a reminder, Tony aims to quickly deliver a working solution for dissatisfied customers, while Johnny focuses on building a more accurate and robust model to replace Tony’s solution when ready), they will approach the model tuning stage with different strategies:

- **Tony** will start by performing <code>30</code> trials for each configuration he wants to explore. If he identifies configurations that struggle to achieve acceptable accuracy, he will perform <code>20</code> additional trials for those configurations.

- **Johnny** will perform <code>100</code> trials for each configuration to allow thorough exploration of the hyperparameter space and maximize the chances of optimal accuracy.

**Configuration Choices**

- **Neural network architecture**  
  Both Tony and Johnny will explore all available architectures:
  - <code>'CLSTMTWB'</code>
  - <code>'CLSTMTFAWB'</code>
  - <code>'STTWB'</code>
  - <code>'STTFAWB'</code>
  - <code>'TCANWB'</code><br><br>

- **Training type**  
  Tony will use only one training type per GPS outages duration:
  - For short GPS outages, he will use <code>random</code> to generalize across beginning and within-session cases.
  - For medium and long outages (which only occur within sessions), he will use <code>centered</code>.<br><br>

  Johnny will use both fixed and random training types:
  - For short outages, he will use <code>random</code>, <code>beginning</code>, and <code>centered</code>.
  - For medium and long outages, he will use <code>random</code> and <code>centered</code>.<br><br>

- **Adversarial examples training**  
  Tony will not use adversarial examples (<code>None</code>).  
  Johnny will explore both without (<code>None</code>) and with on IMU data (<code>'imu'</code>).

- **Coefficient of frequency division**  
  To manage GPS outages up to 10 minutes at a frequency of 100 Hz, both will use a frequency division coefficient of <code>10</code> to reduce computational cost.

- **GPS outage durations**  
  Tony will use:
  - <code>("00:10", "00:30")</code>
  - <code>("00:30", "05:00")</code>
  - <code>("05:00", "10:00")</code>

  Johnny will use:
  - <code>("00:10", "00:30")</code>
  - <code>("00:30", "02:00")</code>
  - <code>("02:00", "05:00")</code>
  - <code>("05:00", "10:00")</code>


### 6.1. Tony

Here is the recap of the configurations that Tony will explore:

| Training type | Architecture | Adversarial examples | GPS outages capacity | Coeff freq division |
| ------------- | ------------ | -------------------- | -------------------- | ------------------- |
| Random        | CLSTMTWB     | None                 | 00:10 to 00:30       | 10                  |
| Random        | CLSTMTFAWB   | None                 | 00:10 to 00:30       | 10                  |
| Random        | STTWB        | None                 | 00:10 to 00:30       | 10                  |
| Random        | STTFAWB      | None                 | 00:10 to 00:30       | 10                  |
| Random        | TCANWB       | None                 | 00:10 to 00:30       | 10                  |
| Centered      | CLSTMTWB     | None                 | 00:30 to 05:00       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 00:30 to 05:00       | 10                  |
| Centered      | STTWB        | None                 | 00:30 to 05:00       | 10                  |
| Centered      | STTFAWB      | None                 | 00:30 to 05:00       | 10                  |
| Centered      | TCANWB       | None                 | 00:30 to 05:00       | 10                  |
| Centered      | CLSTMTWB     | None                 | 05:00 to 10:00       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 05:00 to 10:00       | 10                  |
| Centered      | STTWB        | None                 | 05:00 to 10:00       | 10                  |
| Centered      | STTFAWB      | None                 | 05:00 to 10:00       | 10                  |
| Centered      | TCANWB       | None                 | 05:00 to 10:00       | 10                  |

Tony starts by performing 30 trials for each configuration.

In [ ]:
for gps_outage_duration, training_type in [
    (("00:10", "00:30"), "random"),
    (("00:30", "05:00"), "centered"),
    (("05:00", "10:00"), "centered")
]:
    
    for model_name in ["CLSTMTWB", "CLSTMTFAWB", "STTWB", "STTFAWB", "TCANWB"]:

        project_tony.run_tuning(
            n_experiments=30,
            model_name=model_name,
            use_adversarial=None,
            training_type=training_type,
            coeff_frequency_division=10,
            gps_outage_duration=gps_outage_duration,
            verbosity=1
        )

Then Tony analyzes the tuning results.

For each configuration, he examines the following metrics provided by the framework:

- <code>Velocity loss</code> to observe the model's training performance (note: values are on scaled data and not interpretable in m/s)
- <code>ATE (in m)</code> to evaluate the average error in trajectory reconstruction

In [ ]:
project_tony.display_dashboard()

By reviewing these metrics on the dashboard, he identifies configurations that show poor reconstruction performance. For such configurations, he plans to run 20 additional tuning trials to try and improve their results.

In [ ]:
project_tony.run_tuning(
    n_experiments=50,
    model_name="CLSTMTFAWB",
    use_adversarial=None,
    training_type="centered",
    coeff_frequency_division=10,
    gps_outage_duration=("00:30", "05:00"),
    verbosity=1
)

project_tony.run_tuning(
    n_experiments=50,
    model_name="TCANWB",
    use_adversarial=None,
    training_type="centered",
    coeff_frequency_division=10,
    gps_outage_duration=("00:30", "05:00"),
    verbosity=1
)

project_tony.run_tuning(
    n_experiments=50,
    model_name="TCANWB",
    use_adversarial=None,
    training_type="centered",
    coeff_frequency_division=10,
    gps_outage_duration=("05:00", "10:00"),
    verbosity=1
)

Finally, Tony performs the evaluation of these configurations.

In [ ]:
for gps_outage_duration, training_type in zip(
    [("00:10", "00:30"), ("00:30", "05:00"), ("05:00", "10:00")],
    ["random", "centered", "centered"]
):
    
    for model_name in ["CLSTMTWB", "CLSTMTFAWB", "STTWB", "STTFAWB", "TCANWB"]:

        project_tony.evaluate_configuration(
            model_name=model_name,
            use_adversarial=None,
            training_type=training_type,
            coeff_frequency_division=10,
            gps_outage_duration=gps_outage_duration,
            verbosity=True
        )

For each configuration, he can now examines the following metrics provided by the framework:

- <code>AVE (in m/s)</code>: Average Velocity Error  
- <code>RMVE (%)</code>: Relative Maximum Velocity Error  
- <code>STDVE (in m/s)</code>: Standard Deviation of the Velocity Error  
- <code>ATE (in m)</code>: Average Trajectory Error  
- <code>RMTE (%)</code>: Relative Maximum Trajectory Error  
- <code>STDTE (in m)</code>: Standard Deviation of the Trajectory Error  
- <code>RDE</code>: Relative Distance Error  
- <code>RFTTE (%)</code>: Relative Form Transformed Trajectory Error  
- <code>STE (%)</code>: Scale Transformation Error  
- <code>TTE (in m)</code>: Translation Transformation Error  
- <code>RTE (in deg)</code>: Rotation Transformation Error  

These evaluation results give Tony a clear view of how each configuration is expected to perform in a real-world production setting.

In [ ]:
project_tony.display_dashboard()

Tony can now validate the models tuning stage.

In [ ]:
project_tony.validate_models_tuning()

### 6.2. Johnny

Here is the recap of the configurations that Johnny will explore:

| Training type | Architecture | Adversarial examples | GPS outages capacity | Coeff freq division |
| ------------- | ------------ | -------------------- | -------------------- | ------------------- |
| Random        | CLSTMTWB     | None                 | 00:10 to 00:30       | 10                  |
| Random        | CLSTMTFAWB   | None                 | 00:10 to 00:30       | 10                  |
| Random        | STTWB        | None                 | 00:10 to 00:30       | 10                  |
| Random        | STTFAWB      | None                 | 00:10 to 00:30       | 10                  |
| Random        | TCANWB       | None                 | 00:10 to 00:30       | 10                  |
| Centered      | CLSTMTWB     | None                 | 00:10 to 00:30       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 00:10 to 00:30       | 10                  |
| Centered      | STTWB        | None                 | 00:10 to 00:30       | 10                  |
| Centered      | STTFAWB      | None                 | 00:10 to 00:30       | 10                  |
| Centered      | TCANWB       | None                 | 00:10 to 00:30       | 10                  |
| Beginning     | CLSTMTWB     | None                 | 00:10 to 00:30       | 10                  |
| Beginning     | CLSTMTFAWB   | None                 | 00:10 to 00:30       | 10                  |
| Beginning     | STTWB        | None                 | 00:10 to 00:30       | 10                  |
| Beginning     | STTFAWB      | None                 | 00:10 to 00:30       | 10                  |
| Beginning     | TCANWB       | None                 | 00:10 to 00:30       | 10                  |
| Random        | CLSTMTWB     | imu                  | 00:10 to 00:30       | 10                  |
| Random        | CLSTMTFAWB   | imu                  | 00:10 to 00:30       | 10                  |
| Random        | STTWB        | imu                  | 00:10 to 00:30       | 10                  |
| Random        | STTFAWB      | imu                  | 00:10 to 00:30       | 10                  |
| Random        | TCANWB       | imu                  | 00:10 to 00:30       | 10                  |
| Centered      | CLSTMTWB     | imu                  | 00:10 to 00:30       | 10                  |
| Centered      | CLSTMTFAWB   | imu                  | 00:10 to 00:30       | 10                  |
| Centered      | STTWB        | imu                  | 00:10 to 00:30       | 10                  |
| Centered      | STTFAWB      | imu                  | 00:10 to 00:30       | 10                  |
| Centered      | TCANWB       | imu                  | 00:10 to 00:30       | 10                  |
| Beginning     | CLSTMTWB     | imu                  | 00:10 to 00:30       | 10                  |
| Beginning     | CLSTMTFAWB   | imu                  | 00:10 to 00:30       | 10                  |
| Beginning     | STTWB        | imu                  | 00:10 to 00:30       | 10                  |
| Beginning     | STTFAWB      | imu                  | 00:10 to 00:30       | 10                  |
| Beginning     | TCANWB       | imu                  | 00:10 to 00:30       | 10                  |
| Random        | CLSTMTWB     | None                 | 00:30 to 02:00       | 10                  |
| Random        | CLSTMTFAWB   | None                 | 00:30 to 02:00       | 10                  |
| Random        | STTWB        | None                 | 00:30 to 02:00       | 10                  |
| Random        | STTFAWB      | None                 | 00:30 to 02:00       | 10                  |
| Random        | TCANWB       | None                 | 00:30 to 02:00       | 10                  |
| Centered      | CLSTMTWB     | None                 | 00:30 to 02:00       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 00:30 to 02:00       | 10                  |
| Centered      | STTWB        | None                 | 00:30 to 02:00       | 10                  |
| Centered      | STTFAWB      | None                 | 00:30 to 02:00       | 10                  |
| Centered      | TCANWB       | None                 | 00:30 to 02:00       | 10                  |
| Random        | CLSTMTWB     | imu                  | 00:30 to 02:00       | 10                  |
| Random        | CLSTMTFAWB   | imu                  | 00:30 to 02:00       | 10                  |
| Random        | STTWB        | imu                  | 00:30 to 02:00       | 10                  |
| Random        | STTFAWB      | imu                  | 00:30 to 02:00       | 10                  |
| Random        | TCANWB       | imu                  | 00:30 to 02:00       | 10                  |
| Centered      | CLSTMTWB     | imu                  | 00:30 to 02:00       | 10                  |
| Centered      | CLSTMTFAWB   | imu                  | 00:30 to 02:00       | 10                  |
| Centered      | STTWB        | imu                  | 00:30 to 02:00       | 10                  |
| Centered      | STTFAWB      | imu                  | 00:30 to 02:00       | 10                  |
| Centered      | TCANWB       | imu                  | 00:30 to 02:00       | 10                  |
| Random        | CLSTMTWB     | None                 | 02:00 to 05:00       | 10                  |
| Random        | CLSTMTFAWB   | None                 | 02:00 to 05:00       | 10                  |
| Random        | STTWB        | None                 | 02:00 to 05:00       | 10                  |
| Random        | STTFAWB      | None                 | 02:00 to 05:00       | 10                  |
| Random        | TCANWB       | None                 | 02:00 to 05:00       | 10                  |
| Centered      | CLSTMTWB     | None                 | 02:00 to 05:00       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 02:00 to 05:00       | 10                  |
| Centered      | STTWB        | None                 | 02:00 to 05:00       | 10                  |
| Centered      | STTFAWB      | None                 | 02:00 to 05:00       | 10                  |
| Centered      | TCANWB       | None                 | 02:00 to 05:00       | 10                  |
| Random        | CLSTMTWB     | imu                  | 02:00 to 05:00       | 10                  |
| Random        | CLSTMTFAWB   | imu                  | 02:00 to 05:00       | 10                  |
| Random        | STTWB        | imu                  | 02:00 to 05:00       | 10                  |
| Random        | STTFAWB      | imu                  | 02:00 to 05:00       | 10                  |
| Random        | TCANWB       | imu                  | 02:00 to 05:00       | 10                  |
| Centered      | CLSTMTWB     | imu                  | 02:00 to 05:00       | 10                  |
| Centered      | CLSTMTFAWB   | imu                  | 02:00 to 05:00       | 10                  |
| Centered      | STTWB        | imu                  | 02:00 to 05:00       | 10                  |
| Centered      | STTFAWB      | imu                  | 02:00 to 05:00       | 10                  |
| Centered      | TCANWB       | imu                  | 02:00 to 05:00       | 10                  |
| Random        | CLSTMTWB     | None                 | 05:00 to 10:00       | 10                  |
| Random        | CLSTMTFAWB   | None                 | 05:00 to 10:00       | 10                  |
| Random        | STTWB        | None                 | 05:00 to 10:00       | 10                  |
| Random        | STTFAWB      | None                 | 05:00 to 10:00       | 10                  |
| Random        | TCANWB       | None                 | 05:00 to 10:00       | 10                  |
| Centered      | CLSTMTWB     | None                 | 05:00 to 10:00       | 10                  |
| Centered      | CLSTMTFAWB   | None                 | 05:00 to 10:00       | 10                  |
| Centered      | STTWB        | None                 | 05:00 to 10:00       | 10                  |
| Centered      | STTFAWB      | None                 | 05:00 to 10:00       | 10                  |
| Centered      | TCANWB       | None                 | 05:00 to 10:00       | 10                  |
| Random        | CLSTMTWB     | imu                  | 05:00 to 10:00       | 10                  |
| Random        | CLSTMTFAWB   | imu                  | 05:00 to 10:00       | 10                  |
| Random        | STTWB        | imu                  | 05:00 to 10:00       | 10                  |
| Random        | STTFAWB      | imu                  | 05:00 to 10:00       | 10                  |
| Random        | TCANWB       | imu                  | 05:00 to 10:00       | 10                  |
| Centered      | CLSTMTWB     | imu                  | 05:00 to 10:00       | 10                  |
| Centered      | CLSTMTFAWB   | imu                  | 05:00 to 10:00       | 10                  |
| Centered      | STTWB        | imu                  | 05:00 to 10:00       | 10                  |
| Centered      | STTFAWB      | imu                  | 05:00 to 10:00       | 10                  |
| Centered      | TCANWB       | imu                  | 05:00 to 10:00       | 10                  |

In [ ]:
for gps_outage_duration, training_types in [
    (("00:10", "00:30"), ["random", "centered", "beginning"]), 
    (("00:30", "02:00"), ["random", "centered"]), 
    (("02:00", "05:00"), ["random", "centered"]), 
    (("05:00", "10:00"), ["random", "centered"])
]:

    for training_type in training_types:
    
        for model_name in ["CLSTMTWB", "CLSTMTFAWB", "STTWB", "STTFAWB", "TCANWB"]:

            for use_adversarial in [None, 'imu']:
    
                project_johnny.run_tuning(
                    n_experiments=30,
                    model_name=model_name,
                    use_adversarial=use_adversarial,
                    training_type=training_type,
                    coeff_frequency_division=10,
                    gps_outage_duration=gps_outage_duration,
                    verbosity=1
                )

In [ ]:
for gps_outage_duration, training_types in [
    (("00:10", "00:30"), ["random", "centered", "beginning"]), 
    (("00:30", "02:00"), ["random", "centered"]), 
    (("02:00", "05:00"), ["random", "centered"]), 
    (("05:00", "10:00"), ["random", "centered"])
]:

    for training_type in training_types:
    
        for model_name in ["CLSTMTWB", "CLSTMTFAWB", "STTWB", "STTFAWB", "TCANWB"]:

            for use_adversarial in [None, 'imu']:
    
                project_johnny.evaluate_configuration(
                    model_name=model_name,
                    use_adversarial=use_adversarial,
                    training_type=training_type,
                    coeff_frequency_division=10,
                    gps_outage_duration=gps_outage_duration,
                    verbosity=True
                )

Johnny can now validate the models tuning stage.

In [ ]:
project_johnny.validate_models_tuning()

## 7. Configuration of the models' predictions averaging

Tony and Johnny can now proceed to the next step of their framework preparation journey: configuring the averaging of the models' predictions.

Tony has trained 5 configurations per subcase, so he will choose to configure the averaging for 2, 3, 4, and 5 configurations.

Johnny, on the other hand, has trained 30 configurations per subcase, so he will choose to configure the averaging for 5, 10, 15, 20, 25, and 30 configurations.

In [ ]:
project_tony.averaging_configuration(
    model_selection_levels=[0.4, 0.6, 0.8, 1.0],
    verbosity=True
)
project_johnny.averaging_configuration(
    model_selection_levels=[0.17, 0.34, 0.50, 0.67, 0.84, 1.0],
    verbosity=True
)

## Production

Tony and Johnny can now use the framework to reconstruct position and velocity during GNSS outages in production.

This part of the use case example is coming soon.